Objective: To find the neighbors surrounding center cells 

Input: preproccessed data from 1-29-23_BrownStudy_VectraPreprocessing_private

Output: processed object data with neighborhood profile features attached to the original dataframe 
- no other work has been done 

In [2]:
### original data frame from csv  ####### 
import pandas as pd
import numpy as np 
import seaborn as sns 
import re 
%matplotlib inline
import glob 

location = '1-29_BrownObjectData_PreProcessed_v12.csv'


original_data = pd.read_csv(location) # usecols = reduced_titles) 
#print(original_data)


#### Deal with the non-numerics that might come up when converting back and forth between CSV ####
original_data['XCenter'] = pd.to_numeric(original_data['XCenter'], errors='coerce')
original_data['YCenter'] = pd.to_numeric(original_data['YCenter'], errors='coerce')
original_data['XMin'] = pd.to_numeric(original_data['XMin'], errors='coerce')
original_data['XMax'] = pd.to_numeric(original_data['XMax'], errors='coerce')
original_data['YMin'] = pd.to_numeric(original_data['YMin'], errors='coerce')
original_data['YMax'] = pd.to_numeric(original_data['YMax'], errors='coerce')

original_data = original_data[original_data['XCenter'].apply(lambda x: isinstance(x, (int, float)) or (isinstance(x, str) and x.isnumeric()))]
print(len(original_data))


#### Change all to float; needed for this code to run 

original_data['XCenter'] = original_data['XCenter'].astype(float)
original_data['YCenter'] = original_data['YCenter'].astype(float)
original_data['XMin'] = original_data['XMin'].astype(float)
original_data['XMax'] = original_data['XMax'].astype(float)
original_data['YMin'] = original_data['YMin'].astype(float)
original_data['YMax'] = original_data['YMax'].astype(float)


group_names = original_data.groupby("ImageLocation").groups
grouped = original_data.groupby("ImageLocation")
slide_names = list(group_names.keys())
print(len(slide_names))
header = original_data.head()
header.to_csv('mycellTypes.csv')
cell_types = ['PanCyto+MHCI-_tumor', 'PanCyto+MHCI-_stroma', 'PanCyto+MHCI+_tumor', 'PanCyto+MHCI+_stroma', 'CD8-CD3+IFNy+_tumor', 'CD8-CD3+IFNy+_stroma', 'CD8-CD3+IFNy-_tumor', 'CD8-CD3+IFNy-_stroma', 'CD8+CD3+IFNy+_tumor', 'CD8+CD3+IFNy+_stroma', 'CD8+CD3+IFNy-_tumor', 'CD8+CD3+IFNy-_stroma', 'CD56+CD3-IFNy+_tumor', 'CD56+CD3-IFNy+_stroma', 'CD56+CD3-IFNy-_tumor', 'CD56+CD3-IFNy-_stroma']


cell_neighbors = []
for i in range(len(cell_types)):
                    cell_neighbors.append(cell_types[i] + ' Neighbors')
zeros_df = np.zeros(( len(original_data),len(cell_neighbors)))
print("len zeros df"+str(len(zeros_df)))
zeros_df = pd.DataFrame(zeros_df, columns = cell_neighbors)
print(len(zeros_df))
print(len(original_data))
original_data = pd.concat([original_data, zeros_df], axis=1)
df=original_data
print(original_data)
print(cell_types)
print(original_data[cell_types])
original_data[cell_types] = original_data[cell_types].apply(pd.to_numeric, errors='coerce')

sums_df = pd.DataFrame(columns=cell_types)

for col in cell_types:
    column_sum = original_data[col].sum()
    sums_df[col] = [column_sum]

# Print the sums DataFrame
print(sums_df)

for column_name in cell_types:
    original_data[column_name] = pd.to_numeric(original_data[column_name], errors='coerce')

    

missing_columns = [col for col in cell_types if col not in original_data.columns]
if missing_columns:
    print(f"The following columns are missing in the DataFrame: {', '.join(missing_columns)}")
else:
    print("All columns are present in the DataFrame.")
print(cell_neighbors)

def apply_to_group(df, cell_types,cell_neighbors): #xbound = 100 ybound = 100, radius must be manually entered
    """ Applying func to each row. Meaning for each row, every row on the slide is used/iterated over and passed through the functiions below. 
    This is the base function that can do an entire single slide, any function that iterates over multiple slides or compares groups must go 
    through this function last. 
    """

    def func(row): 
        ''' This function, for a single row will iterate over the entire slide and check for whether the difference between
        the XCenter and YCenter of each cell is within a "neighbor range" for that particular cell, and append its index to 
        a list for later use at gathering information about that cell from the main dataframe.'''
        row_prime = row.filter(items = ['XCenter','YCenter'])
        #print(row_prime)
        #print(row_prime.name)
        group_prime2 = group_prime.drop(row_prime.name)
        #print(len(group_prime)-len(group_prime2))
        diff = np.subtract(group_prime2, row.filter(items=['XCenter', 'YCenter']).transpose()).abs()
        threshold = 200**2
        #print(diff)
        squared_distances = (diff['XCenter'] ** 2 + diff['YCenter'] ** 2)
        #print(squared_distances)
        neighbor_count = squared_distances[squared_distances < threshold].index.tolist()
        #print(neighbor_count)
        return(list(neighbor_count))
    
    
    def func_2(row,cell_types,cell_neighbors):
        ''' Takes the original data function and uses information of neighbor indices and provides rest of information about the 
        neighbors of each cell and adds it to the dataframe on the end in Neighbor Columns'''
        neighbor_info = pd.DataFrame(group, index = neighbor_count[row['Slide Index']])# make a dataframe with only rows included in the index , use slide index because the list used is per group
        #print(len(neighbor_info))
        neighbor_info_numeric = neighbor_info.apply(pd.to_numeric, errors='coerce')
        neighbor_sum = neighbor_info_numeric.sum(axis=0)
        # neighbor_sum = neighbor_info.sum(axis = 0) # sum all columns 
        #print(neighbor_sum)
        row.loc[cell_neighbors] = neighbor_sum.loc[cell_types].to_numpy() # add each values to the 
        return row
    
    



    print(len(df)) # also helps to note where we are when iterating through multiple slides and what the estimate time will be 
    df.insert(0, 'Slide Index', range(0, len(df))) # THIS IS LIKE A SWTICH THAT MUST BE TURNED ON AND OFF, YOU CANT RUN SOMETHING NEW MORE THAN ONCE 
    group_prime = df.filter(items = ['XCenter','YCenter'])
    group = df
    neighbor_count = group_prime.apply(lambda x: func(x), axis = 1) # apply the inner function to every row, obtain the index
    neighbor_count = list(neighbor_count) 
    neighbor_info =  group.apply(lambda row: func_2(row,cell_types,cell_neighbors), axis = 1) # use the list created (now as variable neighbor_count, in the inner function) 
#     print(neighbor_info)
    return neighbor_info


def single_slide_neighbors(df, slide_number= 0): # this entire function could really be replaced by having a column that is "image number" and just numbering them 
    # down the list based on what the image location is, then this is an extra step that can be done earlier 
    ''' This function is meant to take a integer as an input and index from a created list to a slide within the dataframe 
    This is what is then fed to apply_to_group to feed it only a single slidw without having to use the actual image location to call it '''

    group_names = df.groupby("ImageLocation").groups
    grouped = df.groupby("ImageLocation")
    slide_names = list(group_names.keys())
    print(len(slide_names))
    single_slide = grouped.get_group(slide_names[slide_number])
    print(single_slide)

    return single_slide
    
slide_info = single_slide_neighbors(df,0)
print('yay3')
slide_stats = apply_to_group(slide_info,cell_types,cell_neighbors)
print(slide_stats)
slide_stats.to_csv('allradius_multi_200_new_5_9-24_test2.csv')


def create_neighbor_csv(cell_types,cell_neighbors):
    ''' This uses the inner functions that create columns of neighbor cells and exports as a new csv that can be later imported 
    and code can be performed for data summary.'''
    for i in range(0,371): # THIS RANGE MUST BE ADJUSTED IN TERMS OF HOW MANY SLIDES THERE ARE , OR UNIQUE IMAGE LOCATIONS
        slide_number = i
        def single_slide_neighbors(df,slide_number = 1,cell_types = cell_types,cell_neighbors = cell_types):
            ''' This function is meant to take a integer as an input and index from a created list to a slide within the dataframe 
            This is what is then fed to apply_to_group to feed it only a single slidw without having to use the actual image location to call it '''
            group_names = df.groupby("ImageLocation").groups
            grouped = df.groupby("ImageLocation")
            slide_names = list(group_names.keys())
            print('len_slide_names',len(slide_names))
            single_slide = grouped.get_group(slide_names[slide_number])
            return single_slide

        slide_info = single_slide_neighbors(df, slide_number)
        slide_stats = apply_to_group(slide_info,cell_types,cell_neighbors)
        print(slide_stats)
        # slide_stats.to_csv(r"C:\OyinProject\Proj1.csv", index = False)

        with open(r"allradius_multi_200_new_5_9-24_v6.csv",mode = 'a') as f:
            slide_stats.to_csv(f, header=f.tell()==0,index = False)
        single_slide_neighbors(df, i, cell_types,cell_neighbors)
csv_run = create_neighbor_csv(cell_types,cell_neighbors)

import pandas as pd 
#radius200_df = pd.read_csv('allradius200new_11-9-23.csv')
radius30_df = pd.read_csv('allradius_multi_200_new_5_9-24_v2.csv')
#print(radius200_df)
print(radius30_df)
df = radius30_df
print(df)
print(len(df))
grouped = df.groupby("ImageLocation")
group_names = df.groupby("ImageLocation").groups
slide_names = list(group_names.keys())
print(len(slide_names))
print(slide_names)


df_30um = pd.read_csv('allradius_multi_30_new_5_9-24_v4.csv')
df_200um = pd.read_csv('allradius_multi_200_new_5_9-24_v6.csv')


df_30um_renamed = df_30um[cell_neighbors].add_suffix('_30um')
df_200um_renamed = df_200um[cell_neighbors].add_suffix('_200um')

df_30um_renamed['XCenter'] = df_30um['XCenter']
df_30um_renamed['YCenter'] = df_30um['YCenter']
df_30um_renamed['ImageLocation'] = df_30um['ImageLocation']

df_200um_renamed['XCenter'] = df_200um['XCenter']
df_200um_renamed['YCenter'] = df_200um['YCenter']
df_200um_renamed['ImageLocation'] = df_200um['ImageLocation']
# Merging the dataframes
merged_df = pd.merge( df_30um_renamed, df_200um_renamed, on=['XCenter', 'YCenter', 'ImageLocation'])
merged_df = pd.merge( df_30um, merged_df, on=['XCenter', 'YCenter', 'ImageLocation'])



merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]


# To check if the merge was successful
print(merged_df.head())

merged_df.to_csv('5-9-24-BrownData_neighbors_multiradius_merged.csv')


